# Install packages

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install bertopic

In [4]:
!pip install langchain

In [5]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

# Read data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# how to read the data
data_df_small_no_emoji=pd.read_csv('/content/drive/MyDrive/Stuuudia/Magisterka/data_df_small_no_emoji.csv')
data_df_small_no_emoji.head()

,Keywords,MatchedCompanies,NERs,TitleRaw,DescriptionRaw,ContentRaw
0,"{'FromDescription': ['olly alexander', 'singer...",['BBC'],"{'FromDescription': [{'Position': 4, 'Text': '...",BBC is urged to sack UK's Eurovision entrant O...,The BBC is being urged to drop singer Olly Ale...,The BBC is being urged to drop singer Olly Ale...
1,"{'FromDescription': ['accusing israel', 'olly ...",['Eurovision'],"{'FromDescription': [{'Position': 20, 'Text': ...",UK Eurovision act Olly Alexander criticised fo...,"Olly Alexander, the UK's new Eurovision act, h...","Olly Alexander, the UK's new Eurovision act, h..."
2,"{'FromDescription': ['hindley street', 'repres...","[""McDonald's""]","{'FromDescription': [{'Position': 65, 'Text': ...",Shocking sight at McDonald's as staff complain...,A union representative claims they were barred...,A union representative claims they were barred...
3,"{'FromDescription': ['james palace', 'alleged ...",['Goldman Sachs'],"{'FromDescription': [{'Position': 60, 'Text': ...",Prince Andrew 'invited former Goldman Sachs ba...,The Duke invited a gun smuggler and an alleged...,The Duke invited a gun smuggler and an alleged...
4,"{'FromDescription': ['year eve', 'eve parties'...",['Airbnb'],"{'FromDescription': [{'Position': 0, 'Text': '...",Airbnb set to use AI to help identify unauthor...,Airbnb is set to use artificial intelligence t...,Airbnb is set to use artificial intelligence t...


In [3]:
# how to read the data
path_list=('/content/drive/MyDrive/Stuuudia/Magisterka/list_of_contents.txt')
with open(path_list, 'r') as file:
  list_of_contents = file.readlines()
list_of_contents[0]

"The BBC is being urged to drop singer Olly Alexander as its entrant for Eurovision after it emerged he signed a letter calling Israel an 'apartheid regime'. The Years And Years frontman, 33, was unveiled as next year's candidate for the UK during the Strictly Come Dancing final, which aired on the BBC on Saturday. But he now faces having that role stripped from him after he signed a letter from LGBT charity Voices4London which described Israel as an 'apartheid regime' which is trying to 'ethnically cleanse' Palestine. The statement, which was published on October 20, almost two weeks after Hamas' October 7 attack, also says that Israel has 'terrorised' Palestinian people and there is now a 'genocide' taking place 'in real time'. The Conservatives have accused the BBC of 'either a massive oversight or sheer brass neck' for selecting Alexander, while a Jewish charity has called for him to be replaced and for the broadcaster to cut ties with him. The BBC is not planning on taking any act

In [4]:
maximum_length=max([len(sentence.split()) for sentence in list_of_contents])

# BERTopic

https://maartengr.github.io/BERTopic/getting_started/distribution/distribution.html#example

from BERTopic documentation:

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

possible visualizations:
- topic_model.visualize_barchart(top_n_topics=5)
- topic_model.visualize_topics()

In [5]:
from bertopic import BERTopic
from umap import UMAP
from bertopic.representation import MaximalMarginalRelevance
import numpy as np
import tensorflow as tf

"After having calculated our top n words per topic there might be many words that essentially mean the same thing. As a little bonus, we can use bertopic.representation.MaximalMarginalRelevance in BERTopic to diversify words in each topic such that we limit the number of duplicate words we find in each topic." ~BERTopic documentation

In [11]:
# np.random.seed(42)
# tf.random.set_seed(42)


# umap_model = UMAP(n_neighbors=15, n_components=5,
#                   min_dist=0.0, metric='cosine', random_state=42)
# representation_model = MaximalMarginalRelevance(diversity=0.5) #so that the words in topics are diverse
# topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, representation_model=representation_model, umap_model=umap_model)
# topics, probs = topic_model.fit_transform(list_of_contents)

2024-04-22 11:49:33,947 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/335 [00:00<?, ?it/s]

2024-04-22 11:50:56,391 - BERTopic - Embedding - Completed ✓
2024-04-22 11:50:56,394 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-22 11:51:36,439 - BERTopic - Dimensionality - Completed ✓
2024-04-22 11:51:36,441 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-22 11:51:58,832 - BERTopic - Cluster - Completed ✓
2024-04-22 11:51:58,846 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-22 11:52:14,233 - BERTopic - Representation - Completed ✓


In [12]:
# topic_model.save('/content/drive/MyDrive/Stuuudia/Magisterka/BERTopic_model_22_04_13_50', serialization="pickle")

2024-04-22 11:52:20,759 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [6]:
topic_model = BERTopic.load('/content/drive/MyDrive/Stuuudia/Magisterka/BERTopic_model_22_04_13_50')

In [8]:
# look at the most frequent topics
freq = topic_model.get_topic_info(); freq.head(5)
# "-1 refers to all outliers and should typically be ignored"~BERTopic original notebook

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2826,-1_and_to_of_on,"[and, to, of, on, her, at, has, be, its, they]",[Dear Bel I have a ‘boyfriend’ whom I met eigh...
1,0,225,0_musk_elon_platform_accounts,"[musk, elon, platform, accounts, yaccarino, he...",[The head of advertising for NBCUniversal is i...
2,1,211,1_amazons_jassy_employees_bezos,"[amazons, jassy, employees, bezos, its, wareho...",[Jeff Bezos is the founder and executive chair...
3,2,181,2_bp_looney_climate_profits,"[bp, looney, climate, profits, chevron, renewa...",[BP has scaled back its climate change plans a...
4,3,175,3_ai_chatgpt_bing_openai,"[ai, chatgpt, bing, openai, chatbot, microsoft...",[The James Webb space telescope cost $10bn (£8...


In [15]:
topic_model.get_topic(1)  # Select the most frequent topic

[('amazons', 0.014713588438781735),
 ('jassy', 0.00895673671353053),
 ('employees', 0.007650873976183305),
 ('bezos', 0.005766535023043656),
 ('its', 0.005290316786336113),
 ('warehouse', 0.004772515835303871),
 ('delivery', 0.00473641330142851),
 ('layoffs', 0.004489450079029354),
 ('andy', 0.004464660078811957),
 ('seattle', 0.004177119526922513)]

In [26]:
topic_model.topics_[0]#which topic was predicted for row 0 of input data

47

In [19]:
topic_model.topics_[:10] # predicted topics for the first 10 documents

[47, 47, 93, 37, 21, 7, 2, 80, 12, -1]

# How to improve BERTopic
to avoid -1 class e.g. for topic_model.topics_[9]

https://maartengr.github.io/BERTopic/getting_started/distribution/distribution.html

get second most suitable topic

In [7]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(list_of_contents[:10], calculate_tokens=True)
topic_distr[9] ##it includes all topics except from -1

100%|██████████| 1/1 [00:00<00:00,  1.87it/s]


array([0.        , 0.00241215, 0.11940964, 0.        , 0.        ,
       0.00277138, 0.        , 0.        , 0.        , 0.00240268,
       0.        , 0.02866096, 0.        , 0.        , 0.        ,
       0.05522179, 0.01327466, 0.        , 0.23373321, 0.        ,
       0.10695888, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01918708, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00263771,
       0.        , 0.02206162, 0.        , 0.01509666, 0.0047987 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00692333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01580172, 0.00541367,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00724

In [15]:
topic_distr[9].argmax() #so topic 18 is the second most appropriate for the given document

18

# Save BERTopic output for the current data

In [16]:
columns=['TitleRaw', 'ContentRaw', 'MatchedCompanies', 'BERTopicAnswer']
results_df=pd.DataFrame(columns=columns, index=range(data_df_small_no_emoji.shape[0]))
topic_distr, _ = topic_model.approximate_distribution(list_of_contents[:10])
for i in range(data_df_small_no_emoji.shape[0]):
  if i%250==0:
    print(i)
  if topic_model.topics_[i]==-1:
    index=topic_distr[9].argmax()
  else:
    index=topic_model.topics_[i]
  bertopic_answer=topic_model.topic_representations_[index]
  names_list=[item[0] for item in bertopic_answer]
  print(names_list)
  results_df.loc[i]=[data_df_small_no_emoji['TitleRaw'][i], data_df_small_no_emoji['ContentRaw'][i], data_df_small_no_emoji['MatchedCompanies'][i], names_list]
results_df.to_csv('/content/drive/MyDrive/Stuuudia/Magisterka/BERTopic_output_22_04_14_50.csv', index=False)

100%|██████████| 1/1 [00:00<00:00,  3.96it/s]


Streaming output truncated to the last 5000 lines.
['lineker', 'bbc', 'verifyerrors', 'email', 'gary', 'impartiality', 'guidelines', 'asylum', 'mr', 'pa']
['suisse', 'swiss', 'banks', 'takeover', 'billion', 'snb', 'reuters', 'hamers', 'ubss', 'investment']
['tesla', 'selfdriving', 'crash', 'autopilot', 'fires', 'parked', 'vehicles', 'batteries', 'freeway', 'safety']
['swiss', 'collapse', 'banking', 'shares', 'suisses', 'silicon', 'svb', 'wednesday', 'sector', 'ftse']
['at1', 'bondholders', 'suisse', 'swiss', 'ubs', 'equity', 'wiped', 'regulator', 'losses', 'banking']
['republic', 'banks', 'deposits', 'first', 'jpmorgan', 'svb', 'moodys', 'silicon', 'stock', 'signature']
['lineker', 'bbc', 'verifyerrors', 'email', 'gary', 'impartiality', 'guidelines', 'asylum', 'mr', 'pa']
['lineker', 'bbc', 'verifyerrors', 'email', 'gary', 'impartiality', 'guidelines', 'asylum', 'mr', 'pa']
['at1', 'bondholders', 'suisse', 'swiss', 'ubs', 'equity', 'wiped', 'regulator', 'losses', 'banking']
['suisse', 

# sentiment analysis

Max length of the input for BERT is 512, which is a problem, as the input data in actaware dataset is longer.

Trying to solve the issue with https://github.com/ccdv-ai/convert_checkpoint_to_lsg.

In [30]:
!pip install lsg-converter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 2.9 MB/s eta 0:00:00


In [43]:
from transformers import BertForSequenceClassification, BertTokenizer
from lsg_converter import LSGConverter

def run_bert_sentiment_lsg(list_of_contents):
  sentiment_list=[" " for i in range(len(list_of_contents))]
  converter = LSGConverter(max_sequence_length=maximum_length)
  model, tokenizer = converter.convert_from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  # model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  # tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

  i=0
  while i < len(list_of_contents):
    if i%250==0:
      print(i)
    input_ids = tokenizer.encode(list_of_contents[i], return_tensors="pt")
    output=model(input_ids)
    predictions = output.logits.argmax(dim=1)
    sentiment_lab=['1 star', '2 stars', '3 stars', '4 stars', '5 stars']
    sentiment_list[i]=sentiment_lab[predictions.item()]
    i+=1

  # input_ids = tokenizer.encode(text, return_tensors="pt")
  # output=model(input_ids)
  # predictions = output.logits.argmax(dim=1)
  # sentiment_lab=['1 star', '2 stars', '3 stars', '4 stars', '5 stars']
  # return sentiment_lab[predictions.item()]
  return sentiment_list

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from lsg_converter import LSGConverter

def run_bert_sentiment(text):
  model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  input_ids = tokenizer.encode(text, return_tensors="pt")
  output=model(input_ids)
  predictions = output.logits.argmax(dim=1)
  sentiment_lab=['1 star', '2 stars', '3 stars', '4 stars', '5 stars']
  return sentiment_lab[predictions.item()]

In [ ]:
sentiment_list=run_bert_sentiment_lsg(list_of_contents)#crashes after using all available RAM

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of LSGBertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized: ['bert.embeddings.global_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0


In [1]:
sentiment_list=[" " for i in range(len(list_of_contents))]

NameError: name 'list_of_contents' is not defined

In [42]:
import warnings
warnings.filterwarnings("ignore")
i=0
while i < len(list_of_contents):
  if i%250==0:
    print(i)
  sentiment_list[i]=run_bert_sentiment(text=list_of_contents[i]) #przeliczone teraz do 2750
  i+=1
warnings.resetwarnings()

0


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of LSGBertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized: ['bert.embeddings.global_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of LSGBertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized: ['bert.embeddings.global_embeddings.weight']
You should probably TRAIN this mo

KeyboardInterrupt: 

In [39]:
len([elem for elem in sentiment_list if elem !=" "])

38

TO DO:
- puścić całość
- wymyślić jak oceniać wyniki?
- znaleźć na pytanie, czy keywords bertopic sobie sam znajduje w tekście, czy ma jakieś predefined keywords?
- co z topic -1? może dla tamtych brać z `approximate_distribution` drugi najczęstszy?

In [ ]:
#how to get topic (list of words and their probablities) for a given record
topic_model.get_topic(0)

#how to get sentiment
run_bert_sentiment_more_configurations(text="I love movie!")

'5 stars'

In [ ]:
# https://huggingface.co/meta-llama/Llama-2-13b-chat-hf


In [ ]:
from langchain_community.llms import LlamaCpp
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
import os
from torch import cuda
import urllib

HF_AUTH = os.getenv('HF_AUTH', None)
os.environ['HF_HOME'] = os.getenv('HF_HOME', 'models')
os.makedirs(os.getenv('HF_HOME'), exist_ok=True)
model_id='llama-2-7b-chat.Q5_K_S.gguf'
path_to_model = os.path.join(os.getenv('HF_HOME'), model_id)
link_to_model = f"https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/{model_id}"
if not os.path.isfile(path_to_model):
    print(f"--- Downloading {model_id}...")
    urllib.request.urlretrieve(link_to_model, path_to_model)
    print(f"--- Downloaded {model_id} successfully.")
else:
    print(f"--- Model {model_id} already downloaded.")

llama_temperature = 0.5
device = 'cuda' if cuda.is_available() else 'cpu'
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
chunk_size = 400
chunk_overlap = 50

retrieve_top_k_docs_bm25 = 1
retrieve_top_k_docs_vector = 1
context_length_for_llm = chunk_size*(retrieve_top_k_docs_bm25 + retrieve_top_k_docs_vector)+200 #not larger than 2048

n_gpu_layers = -1 if device == 'cuda' else 0
llm = LlamaCpp(
    model_path=path_to_model,
    temperature=llama_temperature,
    max_tokens=min(context_length_for_llm*2, 4096),
    n_gpu_layers=n_gpu_layers,
    n_ctx=min(context_length_for_llm, 2048),
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

--- Downloading llama-2-7b-chat.Q5_K_S.gguf...


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

--- Downloaded llama-2-7b-chat.Q5_K_S.gguf successfully.


llm_load_tensors: offloading 32 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 33/33 layers to GPU
llm_load_tensors:        CPU buffer size =    85.94 MiB
llm_load_tensors:      CUDA0 buffer size =  4349.56 MiB
..................................................................................................
llama_new_context_with_model: n_ctx      = 1024
llama_new_context_with_model: n_batch    = 8
llama_new_context_with_model: n_ubatch   = 8
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:      CUDA0 KV buffer size =   512.00 MiB
llama_new_context_with_model: KV self size  =  512.00 MiB, K (f16):  256.00 MiB, V (f16):  256.00 MiB
llama_new_context_with_model:  CUDA_Host  output buffer size =     0.12 MiB
llama_new_context_with_model:      CUDA0 compute buffer size =     1.53 MiB
llama_new_context_with_model:  CUDA_Host compute buffer size =     0.16 MiB
ll

'TitleRaw', 'ContentRaw', 'MatchedCompanies', 'BERTopicAnswer', 'Sentiment', 'LlamaAnswer'

In [ ]:
# llama_answer=[" " for _ in range(data_df_small_no_none.shape[0])]
columns=['TitleRaw', 'ContentRaw', 'MatchedCompanies', 'BERTopicAnswer', 'Sentiment', 'LlamaAnswer']
results_df=pd.DataFrame(columns=columns, index=range(data_df_small_no_none.shape[0]))
for i in range(data_df_small_no_none.shape[0]):
  # if i>10:
  #   break
  if i%250==0:
    print(i)
  ner=data_df_small_no_none['MatchedCompanies'][i]
  bertopic_answer=topic_model.topic_representations_[topic_model.topics_[i]]
  names_list=[item[0] for item in bertopic_answer]
  bert_answer=sentiment_list[i]
  main_prompt=f"Create short and conceise answer including all of the following information: the name of the company involved: {ner}, main topic of the article: {names_list}, and the sentiment of the text: {bert_answer}. Keep the answer concise. The length of your answer should be exactly 10 words."
  llm_output=llm(main_prompt)
  llm_output=llm_output.replace("\n", " ")
  results_df.loc[i]=[data_df_small_no_none['TitleRaw'][i], data_df_small_no_none['ContentRaw'][i], data_df_small_no_none['MatchedCompanies'][i], names_list, bert_answer, llm_output]

0


Llama.generate: prefix-match hit



The BBC has been accused of being biased against Hamas, with some critics claiming that the broadcaster's editorial stance is anti-Semitic. (BBC, hamas, terrorist, antisemitism, Jews, civilians, staff, attacks, bbcs, editorial, Simpson)
The sentiment of this article is 2 stars.


llama_print_timings:        load time =     264.64 ms
llama_print_timings:      sample time =     121.86 ms /    84 runs   (    1.45 ms per token,   689.33 tokens per second)
llama_print_timings: prompt eval time =     979.50 ms /    85 tokens (   11.52 ms per token,    86.78 tokens per second)
llama_print_timings:        eval time =    2360.94 ms /    83 runs   (   28.45 ms per token,    35.16 tokens per second)
llama_print_timings:       total time =    4651.62 ms /   168 tokens
Llama.generate: prefix-match hit



The company involved is Eurovision, and the main topic of the article is Hamas' use of civilians as human shields during attacks on Israeli staff and BBC editorial. The sentiment of the text is 1 star because it is a negative and critical assessment of Hamas' actions.

...


Answer: David Attenborough's wildlife series filmed on Isles of Scilly sparks outrage over rabbit culling by ANT Group. Sentiment: 1 star.


llama_print_timings:        load time =     264.64 ms
llama_print_timings:      sample time =      25.62 ms /    43 runs   (    0.60 ms per token,  1678.25 tokens per second)
llama_print_timings: prompt eval time =     907.72 ms /    80 tokens (   11.35 ms per token,    88.13 tokens per second)
llama_print_timings:        eval time =    1082.01 ms /    42 runs   (   25.76 ms per token,    38.82 tokens per second)
llama_print_timings:       total time =    2238.04 ms /   122 tokens
Llama.generate: prefix-match hit



The name of the company involved is Amazon, the main topic of the article is toys.


llama_print_timings:        load time =     264.64 ms
llama_print_timings:      sample time =      16.27 ms /    21 runs   (    0.77 ms per token,  1290.40 tokens per second)
llama_print_timings: prompt eval time =     788.62 ms /    71 tokens (   11.11 ms per token,    90.03 tokens per second)
llama_print_timings:        eval time =     510.62 ms /    20 runs   (   25.53 ms per token,    39.17 tokens per second)
llama_print_timings:       total time =    1477.91 ms /    91 tokens
Llama.generate: prefix-match hit



The name of the company involved

In [ ]:
results_df

In [ ]:
results_df.to_csv('/content/drive/MyDrive/Stuuudia/Magisterka/pipeline_results2.csv', index=False)

### przemyślenia:
- jeśli dla pojedynczego dokumentu tzn w moim przypadku newsa dostaję topic, który BERTopic znalazł i ten topic zawiera 10 (?) słów różnych, to będzie ok, bo ja wtedy na podstawie tych słów mogę wygenerować tekst (no i jeszcze używając czegoś do sentiment) llamą i odpowiedź będzie?

In [ ]:
topic_distr, _ = topic_model.approximate_distribution(list_of_contents)

100%|██████████| 11/11 [01:31<00:00,  8.36s/it]


In [ ]:
topic_distr[0]

array([0.05801567, 0.        , 0.0247683 , 0.        , 0.        ,
       0.04126159, 0.00138637, 0.        , 0.00316302, 0.02203856,
       0.        , 0.        , 0.00175138, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00913905, 0.        ,
       0.        , 0.1865974 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00562013,
       0.        , 0.        , 0.22547687, 0.        , 0.        ,
       0.00147208, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00156516, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00137078, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00678332, 0.        ,
       0.        , 0.00504383, 0.        , 0.01109547, 0.     

In [ ]:
topic_model.visualize_distribution(topic_distr[1])